In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [4]:
inputs = tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Preheat the oven to 375 degrees F. In a large bowl, combine the olive']


In [5]:
inputs = tokenizer("Get the Named entity from this sentence: Total revenue is expected to be between $115.0 million and $117.0 million, representing a year-over-year increase of 24% to 27%.", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['san francisco']


In [10]:
advanced_ner = """Microsoft Corporation is a company that makes computer software and video games. Bill Gates and Paul Allen founded the company in 1975 
[Company]: Microsoft, [Founded]: 1975, [Founders]: Bill Gates, Paul Allen 
 
Amazon.com, Inc., known as Amazon , is an American online business and cloud computing company. It was founded on July 5, 1994 by Jeff Bezos 
[Company]: Amazon, [Founded]: 1994, [Founders]: Jeff Bezos 
 
Apple Inc. is a multinational company that makes personal computers, mobile devices, and software. Apple was started in 1976 by Steve Jobs and Steve Wozniak."""
inputs = tokenizer(advanced_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['[Company]: Apple']


In [12]:
report_ner = """"Revenue is $503MN up 77% year over year
[Metric]: Revenue, [Money]: $503MN, [PCT]: 77%, [CALENDAR]: year over year

Revenue is $603MN up 87% year over year
[Metric]: Revenue, [Money]: $603MN, [PCT]: 87%, [CALENDAR]: year over year

Revenue is $63MN down 12% year over year.
"""
inputs = tokenizer(report_ner, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Revenue is $503MN up 77% year over year [Metric]: Revenue,']
